In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# 하이퍼파라미터
EPOCHS = 1000

In [32]:
# layer 설정
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128,
                                        input_dim=2,
                                        activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10,
                                        activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

In [33]:
# 학습 루프 정의
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)    
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)   

### Data

In [34]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
        
pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

In [35]:
# model 생성
model = MyModel()

In [36]:
# loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# optimizer
optimizer = tf.keras.optimizers.Adam()

# Accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

### Training

In [39]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
        template = 'Epoch {}, Loss : {}, Accuracy : {}'
    if epoch % 50 == 0:
        print(template.format(epoch + 1, 
                              train_loss.result(),
                              train_accuracy.result() * 100))

Epoch 1, Loss : 0.34520527720451355, Accuracy : 88.08073425292969
Epoch 51, Loss : 0.33298081159591675, Accuracy : 88.24717712402344
Epoch 101, Loss : 0.3236723840236664, Accuracy : 88.38372802734375
Epoch 151, Loss : 0.31616219878196716, Accuracy : 88.49390411376953
Epoch 201, Loss : 0.3100512623786926, Accuracy : 88.58892822265625
Epoch 251, Loss : 0.305002897977829, Accuracy : 88.67093658447266
Epoch 301, Loss : 0.3006013035774231, Accuracy : 88.74000549316406
Epoch 351, Loss : 0.29696348309516907, Accuracy : 88.80691528320312
Epoch 401, Loss : 0.2936619818210602, Accuracy : 88.86758422851562
Epoch 451, Loss : 0.2908060848712921, Accuracy : 88.92869567871094
Epoch 501, Loss : 0.28814372420310974, Accuracy : 88.98580932617188
Epoch 551, Loss : 0.2858043313026428, Accuracy : 89.04021453857422
Epoch 601, Loss : 0.2836325764656067, Accuracy : 89.09767150878906
Epoch 651, Loss : 0.28166908025741577, Accuracy : 89.14888763427734
Epoch 701, Loss : 0.27987605333328247, Accuracy : 89.1969985

### Save

In [46]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
# transpose 필수
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                    W_h=W_h,
                    b_h=b_h,
                    W_o=W_o,
                    b_o=b_o)